# Basic Convolutional Neural Network

## Convolutional Network Framework

![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

## MNIST Dateset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

In [22]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf

In [23]:
# Import MNIST dataset 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print ("Done......!")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Done......!


In [24]:
# Initialization Parameters
# hyper Params
num_steps = 500
learning_rate = 0.001
batch_size = 128
display_step = 10

# Network Parameters
num_input = 784 # MNIST input features (image shape: 28 * 28)
num_classes = 10 # MNIST total classes (0 - 9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf input Graph
X = tf.placeholder(tf.float32, shape = [None, num_input], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_classes], name = 'Y')
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

In [25]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1],
                         padding = 'SAME')

def conv_net(x, weights, biases, dropout):
    
    # MNISET input is a 1-D vector of 784 features(image shape: 28 * 28)
    # reshape to match picture format [Height * Weight * channels]
    # Tensor input become 4-D: [Batch size, Height, Weight, Channel]
    x = tf.reshape(x, shape = [-1, 28, 28 ,1])
    
    # convolutional layer1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # maxpooling 1
    conv1 = maxpool2d(conv1, k = 2)
    
    # convolutinal layer2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # maxpooling 2
    conv2 = maxpool2d(conv2, k = 2)
    
    # fully connected layer
    fc1 = tf.reshape(conv2, shape = [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # apply dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # output, class prediction
    class_out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return class_out

In [26]:
# Store weights ans biases in layers
weights = {
    # 5 * 5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5 * 5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7 * 7 * 64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class preduction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [27]:
# Construct CNN model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and opitimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
opitimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = opitimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variable
init = tf.global_variables_initializer()

In [28]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    
    for step in range(1, num_steps + 1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run Optimization op
        # backprop
        sess.run(train_op, 
                 feed_dict = {X: batch_x, 
                              Y: batch_y, 
                              keep_prob: dropout})
        
        if step % display_step == 0 or step == 1:
            ls, acc = sess.run([loss_op, accuracy], 
                               feed_dict = {X: batch_x, 
                                            Y: batch_y, 
                                            keep_prob: 1.0})
            
            print ("step : " + str(step) + ", Minibatch Loss = " + \
                  "{:.4f}".format(ls) + ", Training accuracy = " + \
                  "{:.3f}".format(acc))
            
    print("optimizer finished")
    
    # Calculate accuracy for 254 MNISET test images
    print ("Testing Accuracy: " ,\
          sess.run(accuracy, 
                   feed_dict = {X: mnist.test.images[:256], 
                                Y: mnist.test.labels[:256], 
                                keep_prob: 0.1}))
    
        

step : 1, Minibatch Loss = 48038.7773, Training accuracy = 0.070
step : 10, Minibatch Loss = 25846.6016, Training accuracy = 0.305
step : 20, Minibatch Loss = 9372.3301, Training accuracy = 0.531
step : 30, Minibatch Loss = 4507.3320, Training accuracy = 0.734
step : 40, Minibatch Loss = 5405.4043, Training accuracy = 0.742
step : 50, Minibatch Loss = 2583.3276, Training accuracy = 0.836
step : 60, Minibatch Loss = 3133.9741, Training accuracy = 0.844
step : 70, Minibatch Loss = 3582.5566, Training accuracy = 0.820
step : 80, Minibatch Loss = 2127.2993, Training accuracy = 0.859
step : 90, Minibatch Loss = 2126.4131, Training accuracy = 0.883
step : 100, Minibatch Loss = 2396.6968, Training accuracy = 0.875
step : 110, Minibatch Loss = 5044.1992, Training accuracy = 0.844
step : 120, Minibatch Loss = 1833.8031, Training accuracy = 0.836
step : 130, Minibatch Loss = 938.8473, Training accuracy = 0.930
step : 140, Minibatch Loss = 1424.1646, Training accuracy = 0.898
step : 150, Minibatc